# **[Deploy a Multi-Endpoint Model to a real-time inference](https://anishmahapatra.medium.com/how-to-deploy-a-multi-endpoint-model-to-a-real-time-inference-387529845c82)** 

이 튜토리얼에서는 AWS SageMaker를 사용하여 다중 모델 엔드포인트를 단일 실시간 추론으로 배포하는 방법을 알려드립니다. 

SageMaker 추론을 사용하여 합성 주택 가격 예측 데이터 세트에 대해 학습된 이진 분류 XGBoost 모델 세트를 배포합니다. 이 데이터 세트에는 침실 수, 평방 피트, 욕실 수에 대한 세부 정보가 포함되어 있습니다. 이 모델은 단일 위치에 대한 주택 가격을 예측합니다. 귀하는 이러한 모델을 배포하는 역할을 하는 머신러닝 엔지니어입니다. 블로그가 끝날 때까지 다음을 수행할 수 있습니다:

기존 SageMaker Studio 도메인이 없는 경우 SageMaker Studio 도메인을 생성하고 이 자습서의 나머지 부분에 필요한 권한을 추가하는 [AWS CloudFormation](https://us-east-1.console.aws.amazon.com/cloudformation/home?region=us-east-1#/stacks/quickcreate?templateURL=https://sagemaker-sample-files.s3.amazonaws.com/libraries/sagemaker-user-journey-tutorials/CFN-SM-IM-Lambda-catalog.yaml&stackName=CFN-SM-IM-Lambda-Catalog) 템플릿을 실행합니다.

CloudFromation 스택이름을 **CFN-SM-IM-Lambda-Catalog** 로 변경하지 마시고 10분정도 기다리시면 **SageMaker > studio > StudioDomain 선택후 studio-user** 로 시작합니다. 자세한 안내는 [Deploy a Machine Learning Model to a Real-Time Inference Endpoint](https://aws.amazon.com/tutorials/machine-learning-tutorial-deploy-model-to-real-time-inference-endpoint/) 참고하세요. 

- 각각의 학습된 모델 아티팩트로부터 여러 개의 SageMaker 모델 만들기
- 이러한 모델을 제공하기 위해 실시간 엔드포인트를 구성 및 배포합니다.
- 다중 모델 엔드포인트를 호출하여 테스트 데이터를 사용하여 샘플 예측을 실행합니다.

In [3]:
%pip install --upgrade -q aiobotocore

Note: you may need to restart the kernel to use updated packages.


In [4]:
import boto3
import sagemaker
import time

from sagemaker.image_uris import retrieve
from time import gmtime, strftime
from sagemaker.amazon.amazon_estimator import image_uris

sagemaker_session = sagemaker.Session()
default_bucket = sagemaker_session.default_bucket()
write_prefix = "housing-prices-prediction-mme-demo"

region = sagemaker_session.boto_region_name
s3_client = boto3.client("s3", region_name=region)
sm_client = boto3.client("sagemaker", region_name=region)
sm_runtime_client = boto3.client("sagemaker-runtime")
role = sagemaker.get_execution_role()

# S3 locations used for parameterizing the notebook run
read_bucket = "sagemaker-sample-files"
read_prefix = "models/house_price_prediction"
model_prefix = "models/xgb-hpp"

# S3 location of trained model artifact
model_artifacts = f"s3://{default_bucket}/{model_prefix}/"

# Location
location = ['Chicago_IL', 'Houston_TX', 'NewYork_NY', 'LosAngeles_CA']

test_data = [1997, 2527, 6, 2.5, 0.57, 1]

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [5]:
import boto3

# Initialize the s3 resource
s3 = boto3.resource('s3')

# Rest of your code
for i in range(0, 4):
    copy_source = {'Bucket': read_bucket, 'Key': f"{read_prefix}/{location[i]}.tar.gz"}
    bucket = s3.Bucket(default_bucket)
    bucket.copy(copy_source, f"{model_prefix}/{location[i]}.tar.gz")

In [6]:
# Retrieve the SageMaker managed XGBoost image
training_image = retrieve(framework="xgboost", region=region, version="1.3-1")

# Specify an unique model name that does not exist
model_name = "housing-prices-prediction-mme-xgb"
primary_container = {
                     "Image": training_image,
                     "ModelDataUrl": model_artifacts,
                     "Mode": "MultiModel"
                    }

model_matches = sm_client.list_models(NameContains=model_name)["Models"]
if not model_matches:
    model = sm_client.create_model(ModelName=model_name,
                                   PrimaryContainer=primary_container,
                                   ExecutionRoleArn=role)
else:
    print(f"Model with name {model_name} already exists! Change model name to create new")

In [7]:
# Endpoint Config name
endpoint_config_name = f"{model_name}-endpoint-config"

# Create endpoint if one with the same name does not exist
endpoint_config_matches = sm_client.list_endpoint_configs(NameContains=endpoint_config_name)["EndpointConfigs"]
if not endpoint_config_matches:
    endpoint_config_response = sm_client.create_endpoint_config(
                                                                EndpointConfigName=endpoint_config_name,
                                                                ProductionVariants=[
                                                                    {
                                                                        "InstanceType": "ml.m5.xlarge",
                                                                        "InitialInstanceCount": 1,
                                                                        "InitialVariantWeight": 1,
                                                                        "ModelName": model_name,
                                                                        "VariantName": "AllTraffic",
                                                                    }
                                                                ],
                                                                )
else:
    print(f"Endpoint config with name {endpoint_config_name} already exists! Change endpoint config name to create new")

In [8]:
# Endpoint name
endpoint_name = f"{model_name}-endpoint"

endpoint_matches = sm_client.list_endpoints(NameContains=endpoint_name)["Endpoints"]
if not endpoint_matches:
    endpoint_response = sm_client.create_endpoint(
                                                  EndpointName=endpoint_name,
                                                  EndpointConfigName=endpoint_config_name
                                                 )
else:
    print(f"Endpoint with name {endpoint_name} already exists! Change endpoint name to create new")

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
while status == "Creating":
    print(f"Endpoint Status: {status}...")
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
print(f"Endpoint Status: {status}")

Endpoint Status: Creating...
Endpoint Status: Creating...
Endpoint Status: Creating...
Endpoint Status: InService


엔드포인트를 클릭하면 사용량 통계와 사용량을 확인할 수 있습니다.

이제 기억하시겠지만, 예측을 얻고자 하는 샘플 목록을 만들었습니다. 이제 예측을 얻기 위해 목록을 전달하겠습니다.

- location = [‘Chicago_IL’, ‘Houston_TX’, ‘NewYork_NY’, ‘LosAngeles_CA’] <Br>
- test_data = [1997, 2527, 6, 2.5, 0.57, 1] <Br>

In [9]:
# converting the elements in test data to string
payload = ' '.join([str(elem) for elem in test_data])

for i in range (0,4):
    predicted_value = sm_runtime_client.invoke_endpoint(EndpointName=endpoint_name, TargetModel=f"{location[i]}.tar.gz", ContentType="text/csv", Body=payload)
    print(f"Predicted Value for {location[i]} target model:\n ${predicted_value['Body'].read().decode('utf-8')}")


Predicted Value for Chicago_IL target model:
 $[392504.75]
Predicted Value for Houston_TX target model:
 $[387296.5625]
Predicted Value for NewYork_NY target model:
 $[390451.53125]
Predicted Value for LosAngeles_CA target model:
 $[379517.5]


## **Conclusion**
축하합니다! 실시간 추론 엔드포인트에 머신 러닝 모델 배포 자습서를 마쳤습니다.
 

In [ ]:
# Delete model
sm_client.delete_model(ModelName=model_name)

# Delete endpoint configuration
sm_client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)

# Delete endpoint
sm_client.delete_endpoint(EndpointName=endpoint_name)